In [1]:
pip install gradio bs4 requests Pillow tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.2 MB/s eta 

In [7]:
pip install gradio requests beautifulsoup4 Pillow


In [21]:
import gradio as gr
import requests
from bs4 import BeautifulSoup
from PIL import Image

def scrape_links_with_keywords(query, search_type='isch'):
    """
    Scrape links from Google search based on the query.
    Args:
    - query: The search query.
    - search_type: 'isch' for images, 'web' for webpages.
    """
    query = query.replace(' ', '+')
    if search_type == 'web':
        url = f"https://www.google.com/search?q={query}"
    else:
        url = f"https://www.google.com/search?q={query}&tbm=isch"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')

    links = []
    if search_type == 'web':
        for g in soup.find_all('a'):
            href = g.get('href')
            if href and "url?q=" in href and not "webcache" in href:
                clean_link = href.split("url?q=")[1].split("&sa=U")[0]
                if clean_link and "http" in clean_link:
                    links.append(clean_link)
    else:
        for img in soup.find_all('img'):
            src = img.get('src')
            if src and "http" in src:
                links.append(src)

    return links[:5]

def process_data(name, keywords):
    query = f"{name} {keywords}"

    # Fetch webpage links
    web_links = scrape_links_with_keywords(query, search_type='web')
    # Fetch image links
    image_links = scrape_links_with_keywords(query)

    web_results = []
    for link in web_links:
        score = calculate_keyword_score(link, keywords)
        web_results.append(f"Web Link: {link}, Score: {score}")

    image_results = "\n".join(image_links)

    return "\n".join(web_results), image_results

def calculate_keyword_score(link, keywords):
    # Simplified: score based on how many keywords appear in the link
    score = sum(keyword.lower() in link.lower() for keyword in keywords.split())
    return score

# Adjust the Gradio interface to include two output boxes
iface = gr.Interface(fn=process_data,
                     inputs=["text", "text"],
                     outputs=["text", "text"],
                     description="Enter a name and keywords. The system will return the top 5 Google webpage links related to the name and keywords, along with a basic score indicating the relevance of the link to the keywords; and the top 5 Google Image links related to the name and keywords.")

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://96f6a4379e788c1cd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [23]:
import gradio as gr
import requests
from bs4 import BeautifulSoup

def scrape_links_with_keywords(query, search_type='isch'):
    """
    Scrape links from Google search based on the query.
    """
    query = query.replace(' ', '+')
    if search_type == 'web':
        url = f"https://www.google.com/search?q={query}"
    else:
        url = f"https://www.google.com/search?q={query}&tbm=isch"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, 'html.parser')

    links = []
    if search_type == 'web':
        for g in soup.find_all('a'):
            href = g.get('href')
            if href and "url?q=" in href and not "webcache" in href:
                clean_link = href.split("url?q=")[1].split("&sa=U")[0]
                if clean_link and "http" in clean_link:
                    links.append(clean_link)
    else:
        for img in soup.find_all('img'):
            src = img.get('src')
            if src and "http" in src:
                links.append(src)

    return links[:5]

def process_data(name, keywords):
    query = f"{name} {keywords}"
    # Fetch webpage links
    web_links = scrape_links_with_keywords(query, search_type='web')
    web_results = ["Web Link: " + link for link in web_links]

    # Fetch image links
    image_links = scrape_links_with_keywords(query)

    return web_results, image_links

# Adjust Gradio interface to include multiple image outputs
iface = gr.Interface(fn=process_data,
                     inputs=[gr.Textbox(label="Name"), gr.Textbox(label="Keywords")],
                     outputs=[gr.Textbox(label="Webpage Links"), gr.Gallery(label="Images")],
                     description="Enter a name and keywords. The system will return the top 5 Google webpage links related to the name and keywords; and display the top 5 Google Image links related to the name and keywords.")

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://aba1bc81a30d2f5f73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
